# CSGY-6513 Big Data Final Project
The code here is to fetch text data related to stock market from Reddit, and combine them with daily stock return from Yahoo Finance.

In [1]:
from dotenv import load_dotenv
from datetime import datetime
from pytz import timezone
import requests
import json
import time
from psaw import PushshiftAPI
import pickle
import pandas as pd
import yfinance as yf

import re

## 1. Fetch Posts from Reddit through its API.
We spent more than a week to obtain the posts in this year. To save the partial result, we used pickle.

In [2]:
api = PushshiftAPI()

In [3]:
def getPushshiftData(after, before, sub, limit):
    url = 'https://api.pushshift.io/reddit/search/submission/?'+'size=1000' + '&after='+str(after)+'&before='+str(before) + '&subreddit='+str(sub)
    r = requests.get(url)
    if r.status_code != 200:
        time.sleep(5)
        return getPushshiftData(after, before, sub, limit)
    else:
        return json.loads(r.text)['data']

def getCommentsReddit(submissionId):
    url = 'https://api.pushshift.io/reddit/comment/search/?'+'link_id=' + submissionId + '&q=*&limit=1000'
    r = requests.get(url)
    if r.status_code != 200:
        time.sleep(5)
        return getCommentsReddit(submissionId)
    else:
        return json.loads(r.text)['data']

In [8]:
# reddit = []
reddit = pickle.load(open("reddit.pkl", "rb"))
# after = int(datetime(2022, 1, 1).replace(tzinfo=timezone("US/Eastern")).timestamp())
after = pickle.load(open("after.pkl", "rb"))
before = int(datetime(2022, 12, 10).replace(tzinfo=timezone("US/Eastern")).timestamp())
data = getPushshiftData(after, before, "wallstreetbets", 20000)
print(f"Extract data from {datetime.fromtimestamp(after)} to {datetime.fromtimestamp(before)} ...")
timecheck = time.time() - 1000
while len(data)>0:
    for submission in data:
        reddit.append([submission["id"], submission["title"], "", datetime.fromtimestamp(submission["created_utc"])])
        if submission["num_comments"] > 0:
            comments = getCommentsReddit(submission["id"])
            for comment in comments:
                reddit.append([submission["id"], "", comment["body"], datetime.fromtimestamp(comment["created_utc"])])
    after = data[-1]["created_utc"]+1
    if time.time() - timecheck > 900:
        pickle.dump(reddit, open("reddit.pkl", "wb"))
        pickle.dump(after, open("after.pkl", "wb"))
        h, m = str(datetime.fromtimestamp(time.time()).hour), str(datetime.fromtimestamp(time.time()).minute)
        if len(m) == 1:
            m = '0' + m
        curtime = h + ":" + m
        print(f"    {curtime} | Current timestamp: {datetime.fromtimestamp(after)} | #Rows: {len(reddit)}")
        timecheck = time.time()
    data = getPushshiftData(after, before, "wallstreetbets", 20000)
pickle.dump(reddit, open("reddit.pkl", "wb"))

Extract data from 2022-11-11 20:59:34 to 2022-12-09 23:56:00 ...
    11:12 | Current timestamp: 2022-11-12 10:21:57 | #Rows: 1963073
    11:34 | Current timestamp: 2022-11-13 23:55:35 | #Rows: 1973558
    11:58 | Current timestamp: 2022-11-15 11:48:05 | #Rows: 1982561
    12:17 | Current timestamp: 2022-11-16 13:35:16 | #Rows: 1989758
    12:32 | Current timestamp: 2022-11-17 22:05:03 | #Rows: 1998777
    12:49 | Current timestamp: 2022-11-19 13:20:05 | #Rows: 2009866
    13:10 | Current timestamp: 2022-11-21 15:46:51 | #Rows: 2023416
    13:30 | Current timestamp: 2022-11-23 06:06:30 | #Rows: 2036230
    13:51 | Current timestamp: 2022-11-25 08:13:15 | #Rows: 2047117
    14:12 | Current timestamp: 2022-11-27 17:01:23 | #Rows: 2061182
    14:32 | Current timestamp: 2022-11-29 14:57:55 | #Rows: 2073603
    14:54 | Current timestamp: 2022-11-30 23:57:44 | #Rows: 2085813
    15:15 | Current timestamp: 2022-12-02 12:32:45 | #Rows: 2095977
    15:34 | Current timestamp: 2022-12-04 19:50:26 

In [4]:
sep = '¥'
for i in range(len(reddit)):
    for j in [1, 2]:
        reddit[i][j] = re.sub(sep, ' ', reddit[i][j])

In [5]:
reddit_df = pd.DataFrame(reddit, columns=["id", "title", "comment", "timestamp"])
reddit_df["time_key"] = reddit_df["timestamp"].apply(lambda x: x.date())

## 2. Extract the Stock Market Data from Yahoo Finance API.

In [6]:
spx = yf.Ticker("^GSPC")
spx = spx.history(period="5y")
spx["SP500"] = spx["Close"] / spx["Close"].shift(1) - 1
spx["timestamp"] = spx.index
spx["time_key"] = spx["timestamp"].apply(lambda x: x.date)

tsla = yf.Ticker("TSLA")
tsla = tsla.history(period="5y")
tsla["TESLA"] = tsla["Close"] / tsla["Close"].shift(1) - 1
tsla["timestamp"] = tsla.index
tsla["time_key"] = tsla["timestamp"].apply(lambda x: x.date)

## 3. Combine them Together and Export it as CSV File.

In [7]:
reddit_df = reddit_df.merge(spx[["time_key", "SP500"]], how="left", on="time_key")
reddit_df = reddit_df.merge(tsla[["time_key", "TESLA"]], how="left", on="time_key")

In [8]:
reddit_df.tail()

,id,title,comment,timestamp,time_key,SP500,TESLA
2147349,zhjbwk,,Start auto investing into etf’s also - and rid...,2022-12-09 23:57:10,2022-12-09,-0.00735,0.032345
2147350,zhjbwk,,Are those 1 week expiry options? Sorry we don’...,2022-12-09 23:56:53,2022-12-09,-0.00735,0.032345
2147351,zhjbwk,,&gt;If you're looking to maximize your returns...,2022-12-09 23:55:31,2022-12-09,-0.00735,0.032345
2147352,zhjbwk,,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...,2022-12-09 23:55:23,2022-12-09,-0.00735,0.032345
2147353,zhjbwk,,"Nobody tell him.\n\n*I am a bot, and this acti...",2022-12-09 23:55:18,2022-12-09,-0.00735,0.032345


In [ ]:
reddit_df.to_csv("reddit.csv")

In [20]:
import tqdm
text_dic = {}
for i in tqdm.tqdm(range(len(reddit_df))):
    if reddit_df.loc[i, "time_key"] in text_dic.keys():
        text_dic[reddit_df.loc[i, "time_key"]] += " " + reddit_df.loc[i, "title"] + " " + reddit_df.loc[i, "comment"]
    else:
        text_dic[reddit_df.loc[i, "time_key"]] = reddit_df.loc[i, "title"] + " " + reddit_df.loc[i, "comment"]

100%|██████████| 2147354/2147354 [05:58<00:00, 5989.29it/s] 
